In [2]:
import pandas as pd
import ml_metrics as metrics

user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')

In [3]:
user_hist_df['ts'] = pd.to_datetime(user_hist_df['ts'])
user_hist_df['month'] = user_hist_df['ts'].dt.month

In [8]:
test_1 = user_hist_df[user_hist_df['month'] == 6]
test_2 = user_hist_df[user_hist_df['month'] == 7]

train = user_hist_df[~user_hist_df['month'].isin([6, 7])]

In [12]:
print('7 month shape:', test_2.shape, 
      '\n6 month shape:', test_1.shape, 
      '\ntrain shape:', train.shape)

7 month shape: (26243, 5) 
6 month shape: (25729, 5) 
train shape: (294436, 5)


In [38]:
correct_1 = test_1.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()
correct_2 = test_2.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()


In [51]:
top_5_pred = list(train['movie_id'].value_counts()[:5].index)
top_5_pred

[12304, 7735, 15285, 17518, 19026]

In [52]:
metrics.mapk([top_5_pred]*len(correct_1), list(correct_1.values()), 5)

0.004339662447257384

In [53]:
metrics.mapk([top_5_pred]*len(correct_2), list(correct_2.values()), 5)

0.002423093131057733

on LB 0.00111

# random 5

In [55]:
import random

most_popular_100 = list(train.movie_id.value_counts()[:100].index)

preds = dict()

for user in correct_1.keys():
    preds[user] = random.sample(most_popular_100, 5)
    
metrics.mapk(list(preds.values()), list(correct_1.values()), 5)

0.0031814345991561187

In [56]:
preds = dict()

for user in correct_2.keys():
    preds[user] = random.sample(most_popular_100, 5)
    
metrics.mapk(list(preds.values()), list(correct_2.values()), 5)

0.003251158870627897

on LB 0.00218

# Fav serials

In [86]:
fav_serials = train[(train['episode_id'] != 0) & 
                    (train['month'] == 5)].groupby(['user_id', 'movie_id'])['movie_id'].count()
fav_serials = fav_serials[fav_serials>=2].reset_index(name='cnt')

g = fav_serials.groupby(["user_id"]).apply(lambda x: x.sort_values(["cnt"], ascending = False)).reset_index(drop=True)

user_top_5_serials = g.groupby(["user_id"])['movie_id'].apply(lambda x: list(x[:5])).to_dict()

In [89]:
preds = dict()

for user in correct_1.keys():
    try:
        preds[user] = user_top_5_serials[user]
    except KeyError:
        preds[user] = random.sample(most_popular_100, 5)
    
metrics.mapk(list(preds.values()), list(correct_1.values()), 5)

0.027612341772151897

In [90]:
preds = dict()

for user in correct_2.keys():
    try:
        preds[user] = user_top_5_serials[user]
    except KeyError:
        preds[user] = random.sample(most_popular_100, 5)
    
metrics.mapk(list(preds.values()), list(correct_1.values()), 5)

0.0037106188466947965

норм тема брать любимые сериалы из предыдущего месяца

# Ideas

1. посмотреть на каких пользователях чаще ошибаюсь(новых или старых)
2. какие фильмы я чаще всего пропускаю
3. что чаще всего рекомендую ненужное

# models

1. 